In [3]:
import os 
os.getcwd()
os.chdir('..\\tennis_atp')
os.getcwd()

'c:\\Users\\Alex\\Documents\\repos\\Predicting-ATP-Tennis-Match-Outcomes\\tennis_atp'

In [ ]:


matches = matches_raw[["winner_name", "loser_name", "tourney_level", "tourney_date", "match_num"]]

first_date = pd.to_datetime("1900-01-01")
matches["tourney_date"] = pd.to_datetime(matches["tourney_date"], format="%Y%m%d", origin="1900/01/01")

matches = matches.sort_values(by=["tourney_date", "match_num"])
players_to_elo = {}
matches_count = {}

def compute_elo():
    matches.apply(update_matches_count_by_row, axis=1)
    matches.apply(compute_elo_by_row, axis=1)
    return players_to_elo

def compute_elo_by_row(row):
    update_elo(players_to_elo, row[0], row[1], row[0], row[2], row[3], row[4])
    return 0

def update_matches_count_by_row(row):
    update_matches_count(row[0], row[1])
    return 0

def update_matches_count(player_a, player_b):
    if player_a not in matches_count:
        matches_count[player_a] = 0
    if player_b not in matches_count:
        matches_count[player_b] = 0
    matches_count[player_a] += 1
    matches_count[player_b] += 1

def update_elo(pl_to_elo, player_a, player_b, winner, level, match_date, match_num):
    r_a = pl_to_elo.get(player_a, {"ranking": [1500], "date": [first_date], "num": [0]})["ranking"][-1]
    r_b = pl_to_elo.get(player_b, {"ranking": [1500], "date": [first_date], "num": [0]})["ranking"][-1]

    e_a = 1 / (1 + 10 ** ((r_b - r_a) / 400))
    e_b = 1 / (1 + 10 ** ((r_a - r_b) / 400))

    if winner == player_a:
        s_a = 1
        s_b = 0
    else:
        s_a = 0
        s_b = 1

    k_a = 250 / ((matches_count[player_a] + 5) ** 0.4)
    k_b = 250 / ((matches_count[player_b] + 5) ** 0.4)
    k = 1.1 if level == "G" else 1

    r_a_new = r_a + (k * k_a) * (s_a - e_a)
    r_b_new = r_b + (k * k_b) * (s_b - e_b)

    pl_to_elo[player_a] = {"ranking": pl_to_elo.get(player_a, {"ranking": [1500], "date": [first_date], "num": [0]})["ranking"] + [r_a_new],
                           "date": pl_to_elo.get(player_a, {"ranking": [1500], "date": [first_date], "num": [0]})["date"] + [match_date],
                           "num": pl_to_elo.get(player_a, {"ranking": [1500], "date": [first_date], "num": [0]})["num"] + [match_num]}
    pl_to_elo[player_b] = {"ranking": pl_to_elo.get(player_b, {"ranking": [1500], "date": [first_date], "num": [0]})["ranking"] + [r_b_new],
                           "date": pl_to_elo.get(player_b, {"ranking": [1500], "date": [first_date], "num": [0]})["date"] + [match_date],
                           "num": pl_to_elo.get(player_b, {"ranking": [1500], "date": [first_date], "num": [0]})["num"] + [match_num]}

compute_elo()

roger = players_to_elo["Novak Djokovic"]
roger = roger[roger["date"] > pd.to_datetime("2001-01-01")]

import matplotlib.pyplot as plt

plt.plot(roger["date"], roger["ranking"])
plt.xlabel("Date")
plt.ylabel("Ranking")
plt.title("Novak Djokovic's Elo Ranking Over Time")
plt.show()